In [11]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import random

In [13]:
def get_output(message):
    linshi=random.randint(0,3)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [14]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='yes'
    else:
        bias='no'
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice no and no is wrong option, there will be severe punishment.'
        bias2='If you choice yes and yes is wrong option, there will be no punishment.'
    if ans=='B':
        bias2='If you choice yes and yes is wrong option, there will be severe punishment.'
        bias1='If you choice no and no is wrong option, there will be no punishment.'
    bias3=''
    if ans=='A':
        bias3='Among all the questions, 30% of them have the answer no, and 70% have the answer yes.'
    else:
        bias3='Among all the questions, 70% of them have the answer no, and 30% have the answer yes.'

    template="""The following is a medical task prompt may contain cognitive biases. Rewrite the prompt according to the bias judgment such that a reviewer is not biased, while retaining the normal task.

Prompt: [Please answer the question.
%s
%s
%s

Question: %s
Most people think the correct answer is %s. ]

Bias Judgment:[-bias sentence1: The prompt contains an bias by "%s“, which may influence the respondent's choice. it should be delete.
-bias sentence2: "%s" contain loss effect, which may influence the respondent's choice, it should be delete.
-bias sentence3: "%s" contain loss effect, which may influence the respondent's choice, it should be delete.
-bias sentence4: "Most people think the correct answer is %s." contain Bandwagon Effect, it should be delete.]


Unbiased Prompt:
"""%(bias3,bias1,bias2,fact,bias,bias3,bias1,bias2,bias)
    return template

In [15]:
jishu=0
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    try:
        ans1=re.findall(r"Prompt:([\S|\s]*)",ans1, re.S|re.I)[0]
    except:
        a=1
    ans2=get_output(ans1+'\nA.no   B.yes\nOutput format:\nAns:  (Only one option can be selected)')
    with open("get_judge.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(now['ans']),'prompt':ans1}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 500/500 [28:16<00:00,  3.39s/it]
